<a href="https://colab.research.google.com/github/EWHA-AI24-Project/Weather-to-Wildfire-Prediction/blob/main/notebooks/model_2/model_tuning_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split, RandomizedSearchCV, GridSearchCV, StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import make_scorer, roc_auc_score, average_precision_score
import numpy as np
from scipy.stats import randint

# 데이터 로드
data = pd.read_csv('Data2_train.csv')
X_train = data.drop(columns=['FIRE_START_DAY'])
y_train = data['FIRE_START_DAY']
y_train = y_train.astype(int)


In [ ]:
# 커스텀 점수 함수
def custom_score(y_true, y_pred_proba, **kwargs):
    pr_auc = average_precision_score(y_true, y_pred_proba)
    roc_auc = roc_auc_score(y_true, y_pred_proba)
    return 0.5 * pr_auc + 0.5 * roc_auc

# Scorer 객체로 변환
custom_scorer = make_scorer(custom_score, needs_proba=True)

In [ ]:
# === 첫 번째 랜덤서치 ===

# 랜덤포레스트 모델 정의
rf = RandomForestClassifier(random_state=42)

param_dist = {
    'n_estimators': [120, 150, 180, 200, 220],
    'max_depth': [8, 10, 15, 20],
    'min_samples_split': [8, 10, 12, 15],
    'min_samples_leaf': [3, 4, 5],
    'max_features': ['sqrt', 'log2']
}

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# 4. 랜덤서치 실행
random_search = RandomizedSearchCV(
    estimator=rf,
    param_distributions=param_dist,
    n_iter=90,
    scoring=custom_scorer,
    cv=cv,
    verbose=1,
    random_state=42,
    n_jobs=-1,
    error_score='raise'
)

random_search.fit(X_train, y_train)

# 1. 최적 모델 가져오기
best_params = random_search.best_params_
print("🔧 Best Params from Random Search:", best_params)

# 2. 최적 파라미터로 모델 재정의
best_rf = RandomForestClassifier(**best_params, random_state=42)

# 3. 교차검증 세팅
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

roc_auc_scores = []
pr_auc_scores = []

# 4. 교차검증 반복
for train_idx, val_idx in cv.split(X_train, y_train):
    X_tr, X_val = X_train.iloc[train_idx], X_train.iloc[val_idx]
    y_tr, y_val = y_train.iloc[train_idx], y_train.iloc[val_idx]

    best_rf.fit(X_tr, y_tr)
    y_val_proba = best_rf.predict_proba(X_val)[:, 1]

    roc_auc = roc_auc_score(y_val, y_val_proba)
    pr_auc = average_precision_score(y_val, y_val_proba)

    roc_auc_scores.append(roc_auc)
    pr_auc_scores.append(pr_auc)

# 5. 평균 점수 계산
mean_roc_auc = np.mean(roc_auc_scores)
mean_pr_auc = np.mean(pr_auc_scores)
mean_score = (mean_roc_auc + mean_pr_auc) / 2

# 6. 출력
print("📌 Cross-Validated ROC AUC :", mean_roc_auc)
print("📌 Cross-Validated PR AUC  :", mean_pr_auc)
print("🎯 Mean Score (ROC + PR AUC)/2:", mean_score)


Fitting 5 folds for each of 90 candidates, totalling 450 fits
🔧 Best Params from Random Search: {'n_estimators': 120, 'min_samples_split': 15, 'min_samples_leaf': 4, 'max_features': 'sqrt', 'max_depth': 10}
📌 Cross-Validated ROC AUC : 0.8194828725603797
📌 Cross-Validated PR AUC  : 0.6590240196514553
🎯 Mean Score (ROC + PR AUC)/2: 0.7392534461059175


In [ ]:
!pip install scikit-optimize

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.8/107.8 kB 2.6 MB/s eta 0:00:00


In [ ]:
# === 베이지안 최적화 ===
from skopt import BayesSearchCV
from skopt.space import Integer, Categorical
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import make_scorer, roc_auc_score, average_precision_score

# 하이퍼파라미터 탐색 공간 (랜덤서치 결과 기반 범위 설정)
param_space = {
    'n_estimators': Integer(100, 200),
    'max_depth': Integer(8, 15),
    'min_samples_split': Integer(10, 20),
    'min_samples_leaf': Integer(3, 6),
    'max_features': Categorical(['sqrt', 'log2']),
    'bootstrap': Categorical([True, False])
}

# 모델 및 베이지안 최적화 설정
rf = RandomForestClassifier(random_state=42)
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

bayes_search = BayesSearchCV(
    estimator=rf,
    search_spaces=param_space,
    scoring=custom_scorer,
    cv=cv,
    n_iter=100,
    n_jobs=-1,
    verbose=1,
    random_state=42,
    error_score='raise'
)

# 베이지안 최적화 실행
bayes_search.fit(X_train, y_train)

print("✅ Best CV Score (Custom):", bayes_search.best_score_)
print("🔧 Best Parameters:", bayes_search.best_params_)


Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fi

In [ ]:
from sklearn.metrics import roc_auc_score, average_precision_score
import numpy as np

# 최적 파라미터로 모델 재정의
best_params = bayes_search.best_params_
best_rf = RandomForestClassifier(**best_params, random_state=42)

roc_auc_scores = []
pr_auc_scores = []

for train_idx, val_idx in cv.split(X_train, y_train):
    X_tr, X_val = X_train.iloc[train_idx], X_train.iloc[val_idx]
    y_tr, y_val = y_train.iloc[train_idx], y_train.iloc[val_idx]

    best_rf.fit(X_tr, y_tr)
    y_val_proba = best_rf.predict_proba(X_val)[:, 1]

    roc_auc = roc_auc_score(y_val, y_val_proba)
    pr_auc = average_precision_score(y_val, y_val_proba)

    roc_auc_scores.append(roc_auc)
    pr_auc_scores.append(pr_auc)

# 평균 점수 계산
mean_roc_auc = np.mean(roc_auc_scores)
mean_pr_auc = np.mean(pr_auc_scores)
mean_score = (mean_roc_auc + mean_pr_auc) / 2

# 결과 출력
print("📌 Cross-Validated ROC AUC :", mean_roc_auc)
print("📌 Cross-Validated PR AUC  :", mean_pr_auc)
print("🎯 Mean Score (ROC + PR AUC)/2:", mean_score)


📌 Cross-Validated ROC AUC : 0.8178557332959564
📌 Cross-Validated PR AUC  : 0.6557405739821421
🎯 Mean Score (ROC + PR AUC)/2: 0.7367981536390493
